In [2]:
# coding=utf-8
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [124]:
%time
# 2018
columns = []
uni_id_list = []
uni = []
data = pd.DataFrame(data = [[0, 1, 2, 3, 4]],
                    columns = ['uni_id', 'index', 'desc', 'dim', 'measure'])

for state in page.select('p.MsoListParagraph a[href]'): # through regions
    region_url = host + str(year) + '/' + state['href']
    state_page = BS((requests.get(region_url)).content)

    for uni in state_page.select('.blockcontent tr td.inst a[href]'): # through unis
        uni_id = int(uni['href'][12:])
        uni.append(uni.get_text())
        uni_id_list.append(uni_id)
        uni_url = host + '_vpo/' + uni['href']
        uni_page = BS((requests.get(uni_url)).content)
            
        for indicator in uni_page.select('table#analis_dop tr'): # through the last table
            fields = []
            td_num = len(indicator.find_all('td'))
            
            if td_num == 4: # skip headings and not full rows
                for td in indicator.select('td'):
                    fields.append(td.get_text())
                    
                row = pd.DataFrame(data = [[uni_id, fields[0], fields[1], fields[2], fields[3]]],
                                    columns = ['uni_id', 'index', 'desc', 'dim', 'measure'])
                data = data.append(row)

data = data.iloc[3:, :]
data.set_index('uni_id', inplace = True)
data.to_excel('russian_uni.xlsx')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [26]:
url = 'http://www.shanghairanking.com/ARWU2019.html'
r = requests.get(url)
page = BS(r.text, 'html.parser')

data = pd.DataFrame(data = [[0, 1, 2, 3, 4, 5, 6, 7]],
                    columns = ['Rank', 'Uni', 'Alumni', 'Award', 'HiCi', 'N&S', 'PUB', 'PCP'])

for i, uni in enumerate(page.select('table#UniversityRanking tr')):
    if i == 0:
        continue
    else:
        fields = []
        for j, indicator in enumerate(uni.select('td')):
            if j in [0, 1, 5, 6, 7, 8, 9, 10]:
                if j == 1:
                    ind = (indicator.select('a'))[0].text
                else:
                    ind = indicator.text
                fields.append(ind)
        row = pd.DataFrame(data = [[fields[0], fields[1], fields[2], fields[3], fields[4], fields[5], fields[6], fields[7]]],
                                    columns = ['Rank', 'Uni', 'Alumni', 'Award', 'HiCi', 'N&S', 'PUB', 'PCP'])
        data = data.append(row)

data.set_index('Rank', inplace = True)
data.to_excel('ARWU_Ranking_Full.xlsx')